# $\Lambda_B$ Decay Mode
In this notebook we aim to use the analysis techniques we have made and tuned in a complete fashion. In this way we can estimate our sensitibity in the rare mode. We hope this value is similar to that which could actually be achieved but this is not known since we do not have an unblinded rare mode.

However, we will do a similar test in the normalisation mode. If that works well we can reasonably conclude that our system of analyses works well.

In [1]:
# Set enviornment variables
import os
os.environ["ZFIT_DISABLE_TF_WARNINGS"] = "1"
# Suppress some warnings from TensorFlow related to zfit

In [2]:
import tensorflow as tf
import zfit
from zfit import z
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use([hep.style.ROOT, hep.style.firamath])

2022-03-30 15:03:17.417621: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-30 15:03:17.417725: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Load Data
Ordinarily we would use the standard create_data framework to generate our data for us. In this rare mode case it has already been done for us, yay!

In [7]:
test = pd.read_csv('../../../mnt/d/University/Y4/Project/localproject/9.0.4/test.csv', index_col=[0])
test.head()

,JPs_FD_ORIVX,LStar_ORIVX_CHI2,Lb_ENDVERTEX_CHI2,LN_Lb_IPCHI2_OWNPV,Lb_PT,LN_Lb_MINIPCHI2,LN_Lb_IP01,LN_Lb_IP23,LN_Lb_IP_OWNPV,p_TRACK_VeloCHI2NDOF,...,JPs_IP_TOPPV,LStar_IP_ORIVX,JPs_IP_ORIVX,LN_SUM_CONEISO,LN_SUM_IPCHI2,LN_COS_THETA,LN_COS_LBDIRA,Lb_M,IsSimulated,category
11893,-0.036017,-0.670855,-0.670855,-1.045046,-1.156758,-1.045046,-0.567977,-0.698670,-0.895587,-0.354942,...,-0.600174,-0.153343,-0.203291,-0.720856,-1.453840,-0.339975,-0.261512,4748.478072,False,0
23125,-0.039124,-0.984681,-0.984681,-1.664390,0.529478,-1.664390,1.559475,1.671400,-1.705403,0.445039,...,1.965093,-0.199839,-0.256353,0.102694,1.138569,-0.418031,0.522607,5579.733865,True,1
8405,-0.040081,-0.802956,-0.802956,0.980641,-1.309454,0.980641,0.237914,-0.488809,1.144078,-1.163024,...,-0.534112,0.193383,-0.256515,-0.496455,-0.719871,-2.920263,-0.465136,4702.065504,False,0
11765,-0.040122,-0.825895,-0.825895,-0.994518,0.464034,-0.994518,0.243058,-0.678556,-1.185946,0.417178,...,-0.594549,-0.180973,-0.251228,1.073226,-1.500145,0.154602,0.504919,5614.227106,True,1
23326,-0.036608,0.145000,0.145000,0.079730,-0.242803,0.079730,-1.691390,-1.262434,-0.075864,-0.674160,...,-0.723857,-0.006472,-0.156687,-0.571091,-0.092846,0.147660,0.402862,5013.680684,False,0


# Inspect Data
To gauge how good our techniques are it is first good to know how many signal and how many background events there are in these data.

In [5]:
print(f"There are {test.category.value_counts()[0]} background events")
print(f"There are {test.category.value_counts()[1]} signal events")

There are 3731 background events
There are 3665 signal events
